In [ ]:
# importing necessary libraries

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from random import randint, shuffle
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn import preprocessing
import pandas as pd

In [ ]:
# creation of neural network model
 
def base_model(n):
  model = Sequential([
                    tf.keras.layers.Dense(units = 100,activation = 'relu',input_shape = (2,)),
                    tf.keras.layers.Dense(units = 50,activation = 'relu'),
                    tf.keras.layers.Dense(units = n,activation = 'softmax')
                    ])
  return model

In [ ]:
# Predict lane segment
def predictsegment(model , testing):
  predictions = model.predict(testing,batch_size = 10, verbose = 1)
  return predictions

In [ ]:
# Load dataset and scaling

df = pd.read_csv("/home/sample2.csv")
x = df[['Latitude','Longitude']]
y = df.filter(['Segment'],axis = 1)
x = preprocessing.StandardScaler().fit(x).transform(x)
x_train, x_test, y_train, y_test = tts(x,y,test_size = 0.2,random_state = 10)
x_test1 = x_test

In [ ]:
print(x)

[[-0.40322782 -0.17902528]
 [-0.40322782 -0.15344812]
 [-0.40322782 -0.12787097]
 ...
 [ 0.3998211  -0.01677459]
 [ 0.3998211   0.00880256]
 [ 0.3998211   0.03437972]]


In [ ]:
# creation 

model = base_model(6)
df.head()
# print(x)

# print(y)
# give input as number of output segments in each route

,Latitude,Longitude,Segment
0,-111.119192,31.828087,0
1,-111.119192,31.828097,0
2,-111.119192,31.828107,0
3,-111.119192,31.828117,0
4,-111.119192,31.828127,0


In [ ]:
# Model runner
model.compile(optimizer = Adam(learning_rate = 0.001), loss = "sparse_categorical_crossentropy",metrics = ['accuracy'])
history = model.fit(x_train,y_train, batch_size = 10,epochs = 25,verbose = 1,validation_split=0.2)
model.summary()

Epoch 1/25
1022/1022 [==============================] - 3s 2ms/step - loss: 0.3226 - accuracy: 0.8899 - val_loss: 0.1332 - val_accuracy: 0.9487
Epoch 2/25
1022/1022 [==============================] - 2s 2ms/step - loss: 0.1261 - accuracy: 0.9496 - val_loss: 0.1179 - val_accuracy: 0.9495
Epoch 3/25
1022/1022 [==============================] - 2s 2ms/step - loss: 0.1115 - accuracy: 0.9530 - val_loss: 0.1029 - val_accuracy: 0.9558
Epoch 4/25
1022/1022 [==============================] - 2s 2ms/step - loss: 0.1036 - accuracy: 0.9558 - val_loss: 0.0982 - val_accuracy: 0.9565
Epoch 5/25
1022/1022 [==============================] - 3s 3ms/step - loss: 0.0981 - accuracy: 0.9571 - val_loss: 0.1000 - val_accuracy: 0.9546
Epoch 6/25
1022/1022 [==============================] - 2s 2ms/step - loss: 0.0942 - accuracy: 0.9613 - val_loss: 0.0894 - val_accuracy: 0.9601
Epoch 7/25
1022/1022 [==============================] - 2s 2ms/step - loss: 0.0887 - accuracy: 0.9631 - val_loss: 0.1044 - val_accuracy:

In [ ]:
# Checking accuracy and loss of the model

score = model.evaluate(x_test, y_test, batch_size = 10, verbose =1)
predictions = predictsegment(model , x_test)
# y_test.head(100)
arrpoints = predictions

# print(predictions)
print("Test Loss : " , score[0])
print("Test Accuracy : ", score[1])
print(score)

320/320 [==============================] - 0s 1ms/step
Test Loss :  0.07911230623722076
Test Accuracy :  0.9633458852767944
[0.07911230623722076, 0.9633458852767944]


In [ ]:
# Different types of segment
df['Segment'].value_counts()

5    6496
1    3808
2    2016
0    1848
3    1288
4     504
Name: Segment, dtype: int64

In [ ]:
import json
import numpy as np

In [ ]:
# function to calculate distance between gps data (latitude and longitude) and route data 

Radius = 6371e3

def distanceBetween(point1, point2):
    phi1 = point1[0] * np.pi / 180
    phi2 = point2[0] * np.pi / 180
    deltaPhi = (point2[0] - point1[0]) * np.pi / 180
    deltaLambda = (point2[1] - point1[1]) * np.pi / 180

    a = np.sin(deltaPhi/2) * np.sin(deltaPhi/2) + np.cos(phi1) * np.cos(phi2) * np.sin(deltaLambda/2) * np.sin(deltaLambda/2)

    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    return Radius * c

# fuction which takes the point and which segment it belongs to
def findMatchedPoint(point, segment):
    file = open('/home/route.json', 'r')
    FileData = json.load(file)
    coordinates = []
    for feature in FileData['features']:
        if feature['properties']['segment'] == segment:
            coordinates.extend(feature['geometry']['coordinates'])
            break
    min = distanceBetween(coordinates[0], point)
    minIndex = 0
    for idx in range(1,len(coordinates)):
        dist = distanceBetween(coordinates[idx], point)
        if(dist < min):
            min = dist
            minIndex = idx
    return coordinates[minIndex]

In [ ]:
# point and its snapped location

l = []
for i in range(len(arrpoints)):
  l.append(np.argmax(arrpoints[i]))
print(l)
c = 0
for i in x_test1:
  point = i
  print("Point : " , point , "Point Snapped : " , findMatchedPoint(point, l[c]))
  c+=1

[2, 1, 5, 1, 5, 0, 4, 1, 0, 2, 5, 5, 5, 5, 2, 3, 5, 5, 0, 3, 1, 1, 5, 2, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 4, 1, 2, 5, 5, 5, 2, 0, 5, 1, 5, 5, 1, 1, 2, 1, 1, 0, 3, 0, 2, 1, 5, 0, 5, 3, 5, 5, 1, 3, 2, 0, 5, 0, 5, 5, 0, 5, 5, 4, 5, 5, 5, 5, 5, 1, 5, 5, 2, 5, 0, 3, 5, 4, 1, 3, 5, 5, 5, 5, 5, 1, 2, 1, 5, 1, 5, 5, 2, 5, 2, 5, 5, 5, 5, 5, 3, 1, 0, 5, 0, 5, 5, 5, 5, 1, 5, 0, 5, 3, 5, 5, 5, 5, 5, 5, 1, 2, 3, 2, 5, 5, 0, 2, 5, 5, 1, 2, 0, 1, 0, 3, 5, 5, 5, 5, 5, 5, 5, 0, 5, 0, 3, 5, 2, 0, 0, 5, 2, 5, 5, 5, 3, 1, 2, 3, 5, 5, 5, 1, 3, 3, 5, 1, 1, 1, 3, 0, 5, 5, 3, 3, 1, 5, 5, 3, 5, 0, 3, 5, 5, 2, 1, 1, 5, 0, 1, 1, 0, 3, 1, 5, 1, 1, 5, 5, 0, 2, 2, 5, 2, 0, 2, 5, 5, 2, 3, 5, 5, 5, 0, 5, 5, 3, 1, 5, 0, 0, 5, 0, 1, 5, 1, 2, 2, 0, 5, 5, 2, 1, 5, 5, 5, 2, 2, 3, 5, 3, 5, 5, 1, 5, 3, 0, 5, 5, 1, 5, 1, 4, 5, 1, 5, 5, 1, 3, 5, 5, 5, 2, 5, 5, 4, 5, 1, 2, 5, 5, 5, 5, 0, 1, 4, 0, 5, 4, 3, 1, 5, 5, 1, 2, 1, 5, 1, 5, 1, 5, 1, 1, 4, 5, 1, 5, 0, 5, 3, 0, 0, 2, 2, 0, 0, 2, 0, 3, 2, 1, 1, 5, 5, 5, 5, 5, 1, 2, 1, 1, 1, 

In [ ]:
arr1 = arrpoints

In [ ]:
print(arr1)

[[1.0345093e-12 5.1340750e-05 9.9994826e-01 4.0358154e-07 1.7626424e-19
  8.4192298e-10]
 [8.4801222e-06 9.9999058e-01 1.0033501e-06 3.8542455e-25 7.6499405e-35
  7.1261571e-26]
 [6.4632601e-09 2.4964557e-21 1.9383700e-20 1.8165194e-18 4.6014195e-12
  1.0000000e+00]
 ...
 [9.9924326e-01 7.2385487e-04 3.0905143e-05 3.1918156e-14 1.9368579e-21
  2.0106916e-06]
 [1.8545641e-10 1.7597403e-23 5.3088114e-24 2.0270795e-23 2.0342289e-16
  1.0000000e+00]
 [1.3252727e-11 4.9082752e-24 6.2798901e-23 4.2247545e-20 5.0444116e-10
  1.0000000e+00]]


In [ ]:
print(x_test)

[[ 1.15942026 -0.34596211]
 [ 0.69319052 -1.4701596 ]
 [-1.60912031  0.99536549]
 ...
 [ 0.17676776 -0.67610143]
 [-2.01077044  1.00516982]
 [-1.77276609  1.33150482]]
